In [1]:
import re
import random
from typing import List, Tuple
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


In [2]:
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# "rnn", "lstm" ou "gru"
RNN_KIND = "lstm"
BIDIRECTIONAL = True
EMBED_DIM = 64
HIDDEN_DIM = 64
DROPOUT = 0.2
BATCH_SIZE = 16
EPOCHS = 12
LR = 1e-3
MIN_FREQ = 1
MAX_VOCAB = 20000


In [3]:
# rótulo 1 = positivo, 0 = negativo
positive = [
    "Adorei esse filme!",
    "O jantar estava delicioso.",
    "Hoje acordei muito feliz.",
    "Que dia maravilhoso!",
    "Esse livro é fantástico.",
    "Estou animado para a viagem.",
    "Você é incrível!",
    "A apresentação foi ótima.",
    "Esse lugar é lindo.",
    "Gostei bastante da música.",
    "O atendimento foi excelente.",
    "Estou muito satisfeito com o serviço.",
    "Foi uma experiência maravilhosa.",
    "A comida estava muito boa.",
    "Que surpresa agradável!",
    "O trabalho ficou perfeito.",
    "Estou muito orgulhoso de você.",
    "Esse hotel é excelente.",
    "A paisagem é deslumbrante.",
    "Adorei a festa ontem.",
    "O produto superou minhas expectativas.",
    "Me diverti muito com vocês.",
    "Foi um dia inesquecível.",
    "O show foi espetacular.",
    "Essa notícia me deixou feliz.",
    "Estou muito motivado.",
    "O clima está agradável hoje.",
    "Gostei muito da conversa.",
    "Foi uma compra excelente.",
    "O resultado foi positivo.",
    "A música é maravilhosa.",
    "Que bom receber essa mensagem.",
    "O café está delicioso.",
    "Você fez um ótimo trabalho.",
    "Estou radiante de alegria.",
    "Foi uma decisão acertada.",
    "Que ótimo presente!",
    "Aproveitei muito o passeio.",
    "Estou contente com o progresso.",
    "O filme me emocionou.",
    "Foi um ótimo investimento.",
    "Estou em paz.",
    "Que ótima oportunidade.",
    "Esse restaurante é excelente.",
    "Gostei bastante do curso.",
    "As férias foram maravilhosas.",
    "Estou muito grato.",
    "A viagem foi incrível.",
    "Essa série é muito boa.",
    "Estou muito satisfeito.",
    "Tudo correu bem.",
    "Fiquei impressionado com o trabalho.",
    "O carro é ótimo.",
    "Foi um dia produtivo.",
    "Amei a decoração.",
    "Esse lugar é especial.",
    "Estou empolgado com o futuro.",
    "Você me fez sorrir.",
    "A experiência foi positiva.",
    "Essa música me alegra.",
    "Foi um momento único.",
    "Estou cheio de esperança.",
    "Que ótima surpresa.",
    "Estou satisfeito com o resultado.",
    "Foi uma escolha acertada.",
    "Gostei da solução.",
    "Estou feliz com a conquista.",
    "Foi uma manhã agradável.",
    "Estou otimista.",
    "Essa notícia é maravilhosa.",
    "Que ideia genial!",
    "Adorei a iniciativa.",
    "Você é muito talentoso.",
    "Estou contente com a resposta.",
    "Foi um ótimo aprendizado.",
    "Estou alegre.",
    "O clima está perfeito.",
    "A equipe é excelente.",
    "Estou satisfeito com o atendimento.",
    "Foi um prazer participar.",
    "Gostei da experiência.",
    "Estou confiante.",
    "Esse produto é ótimo.",
    "Estou orgulhoso do resultado.",
    "A nota foi excelente.",
    "Fiquei muito contente.",
    "Esse filme é emocionante.",
    "Estou animado com o projeto.",
    "Foi uma boa surpresa.",
    "O evento foi ótimo.",
    "Estou de bom humor.",
    "Foi uma conquista importante.",
    "Adorei conhecer você.",
    "Estou agradecido.",
    "Esse aplicativo é ótimo.",
    "Gostei da novidade.",
    "Estou muito animado.",
    "O jogo foi emocionante.",
    "Que alegria!",
    "Foi maravilhoso estar aqui."
]

In [4]:
negative = [
    "Detestei esse filme.",
    "A comida estava ruim.",
    "Hoje acordei triste.",
    "Que dia horrível.",
    "Esse livro é péssimo.",
    "Estou desanimado.",
    "Você me decepcionou.",
    "A apresentação foi fraca.",
    "Esse lugar é feio.",
    "Não gostei da música.",
    "O atendimento foi terrível.",
    "Estou muito insatisfeito com o serviço.",
    "Foi uma experiência ruim.",
    "A comida estava sem gosto.",
    "Que surpresa desagradável.",
    "O trabalho ficou malfeito.",
    "Estou decepcionado com você.",
    "Esse hotel é horrível.",
    "A paisagem não é bonita.",
    "Não gostei da festa ontem.",
    "O produto me decepcionou.",
    "Não me diverti nada.",
    "Foi um dia perdido.",
    "O show foi fraco.",
    "Essa notícia me deixou triste.",
    "Estou desmotivado.",
    "O clima está ruim hoje.",
    "Não gostei da conversa.",
    "Foi uma compra péssima.",
    "O resultado foi negativo.",
    "A música é ruim.",
    "Não gostei dessa mensagem.",
    "O café está frio.",
    "Você fez um trabalho ruim.",
    "Estou frustrado.",
    "Foi uma decisão errada.",
    "Que presente horrível.",
    "Não aproveitei o passeio.",
    "Estou descontente com o progresso.",
    "O filme foi chato.",
    "Foi um péssimo investimento.",
    "Estou em conflito.",
    "Que oportunidade perdida.",
    "Esse restaurante é ruim.",
    "Não gostei do curso.",
    "As férias foram cansativas.",
    "Estou muito irritado.",
    "A viagem foi decepcionante.",
    "Essa série é entediante.",
    "Estou insatisfeito.",
    "Tudo deu errado.",
    "Fiquei decepcionado com o trabalho.",
    "O carro é ruim.",
    "Foi um dia improdutivo.",
    "Não gostei da decoração.",
    "Esse lugar é desconfortável.",
    "Estou preocupado com o futuro.",
    "Você me fez chorar.",
    "A experiência foi negativa.",
    "Essa música me entristece.",
    "Foi um momento ruim.",
    "Estou sem esperança.",
    "Que péssima surpresa.",
    "Estou insatisfeito com o resultado.",
    "Foi uma escolha errada.",
    "Não gostei da solução.",
    "Estou triste com a derrota.",
    "Foi uma manhã ruim.",
    "Estou pessimista.",
    "Essa notícia é péssima.",
    "Que ideia ruim.",
    "Não gostei da iniciativa.",
    "Você não tem talento.",
    "Estou descontente com a resposta.",
    "Foi um aprendizado ruim.",
    "Estou desanimado.",
    "O clima está péssimo.",
    "A equipe é ruim.",
    "Estou insatisfeito com o atendimento.",
    "Foi um desprazer participar.",
    "Não gostei da experiência.",
    "Estou inseguro.",
    "Esse produto é ruim.",
    "Estou envergonhado com o resultado.",
    "A nota foi péssima.",
    "Fiquei muito chateado.",
    "Esse filme é entediante.",
    "Estou desanimado com o projeto.",
    "Foi uma péssima surpresa.",
    "O evento foi ruim.",
    "Estou de mau humor.",
    "Foi uma derrota importante.",
    "Não gostei de conhecer você.",
    "Estou ressentido.",
    "Esse aplicativo é ruim.",
    "Não gostei da novidade.",
    "Estou desmotivado.",
    "O jogo foi chato.",
    "Que tristeza.",
    "Foi horrível estar aqui."
]

In [5]:
def split_data(pos: List[str], neg: List[str], train_ratio=0.8) -> Tuple[List[Tuple[str,int]], List[Tuple[str,int]]]:
    data = [(s,1) for s in pos] + [(s,0) for s in neg]
    random.shuffle(data)
    n = int(len(data)*train_ratio)
    return data[:n], data[n:]

train_data, val_data = split_data(positive, negative, train_ratio=0.8)


In [6]:
def tokenize(text: str) -> List[str]:
    text = text.lower()
    # separa por tokens alfanuméricos simples
    return re.findall(r"[a-záàâãéêíóôõúç0-9]+", text)

PAD, UNK = "<pad>", "<unk>"

def build_vocab(samples: List[Tuple[str,int]], min_freq=1, max_size=20000):
    from collections import Counter
    cnt = Counter()
    for s, _ in samples:
        cnt.update(tokenize(s))
    words = [w for w,f in cnt.items() if f >= min_freq]
    words = sorted(words, key=lambda w: (-cnt[w], w))[:max_size-2]
    stoi = {PAD:0, UNK:1}
    for i,w in enumerate(words, start=2):
        stoi[w] = i
    itos = {i:w for w,i in stoi.items()}
    return stoi, itos

stoi, itos = build_vocab(train_data, MIN_FREQ, MAX_VOCAB)
VOCAB_SIZE = len(stoi)

def encode(text: str, stoi: dict) -> List[int]:
    return [stoi.get(tok, stoi[UNK]) for tok in tokenize(text)]


In [7]:
class TextClsDataset(Dataset):
    def __init__(self, pairs: List[Tuple[str,int]], stoi: dict):
        self.samples = pairs
        self.stoi = stoi
    def __len__(self):
        return len(self.samples)
    def __getitem__(self, idx):
        s, y = self.samples[idx]
        return encode(s, self.stoi), y

def collate_fn(batch):
    xs, ys = zip(*batch)
    lengths = [len(x) for x in xs]
    maxlen = max(lengths)
    pad_id = stoi[PAD]
    padded = []
    for x in xs:
        if len(x) < maxlen:
            x = x + [pad_id]*(maxlen - len(x))
        padded.append(x)
    x_tensor = torch.tensor(padded, dtype=torch.long)
    y_tensor = torch.tensor(ys, dtype=torch.long)
    lengths_tensor = torch.tensor(lengths, dtype=torch.long)
    return x_tensor, lengths_tensor, y_tensor

train_loader = DataLoader(TextClsDataset(train_data, stoi), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(TextClsDataset(val_data,   stoi), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

In [8]:

class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, rnn_kind="lstm", bidirectional=True, dropout=0.2, num_classes=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.rnn_kind = rnn_kind.lower()
        self.bidirectional = bidirectional
        self.num_directions = 2 if bidirectional else 1

        if self.rnn_kind == "rnn":
            self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True, bidirectional=bidirectional)
        elif self.rnn_kind == "gru":
            self.rnn = nn.GRU(embed_dim, hidden_dim, batch_first=True, bidirectional=bidirectional)
        else:
            self.rnn = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=bidirectional)

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * self.num_directions, num_classes)

    def forward(self, x, lengths):
        emb = self.embedding(x)  # (B, T, E)
        packed = nn.utils.rnn.pack_padded_sequence(emb, lengths.cpu(), batch_first=True, enforce_sorted=False)
        if self.rnn_kind == "lstm":
            packed_out, (h_n, c_n) = self.rnn(packed)
        else:
            packed_out, h_n = self.rnn(packed)

        # h_n: (num_layers * num_directions, B, hidden)
        if self.bidirectional:
            # pega as duas últimas fatias referentes às duas direções da última camada
            h_fwd = h_n[-2,:,:]
            h_bwd = h_n[-1,:,:]
            h = torch.cat([h_fwd, h_bwd], dim=1)
        else:
            h = h_n[-1,:,:]

        h = self.dropout(h)
        logits = self.fc(h)  # (B, num_classes)
        return logits

model = RNNClassifier(
    vocab_size=VOCAB_SIZE,
    embed_dim=EMBED_DIM,
    hidden_dim=HIDDEN_DIM,
    rnn_kind=RNN_KIND,
    bidirectional=BIDIRECTIONAL,
    dropout=DROPOUT,
    num_classes=2
).to(DEVICE)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

def accuracy(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, lengths, y in loader:
            x, lengths, y = x.to(DEVICE), lengths.to(DEVICE), y.to(DEVICE)
            logits = model(x, lengths)
            pred = logits.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    return correct / total if total > 0 else 0.0

for epoch in range(1, EPOCHS+1):
    model.train()
    running_loss = 0.0
    for x, lengths, y in train_loader:
        x, lengths, y = x.to(DEVICE), lengths.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        logits = model(x, lengths)
        loss = criterion(logits, y)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()
        running_loss += loss.item() * y.size(0)

    train_acc = accuracy(model, train_loader)
    val_acc = accuracy(model, val_loader)
    print(f"Epoch {epoch:02d} | Loss {running_loss/len(train_loader.dataset):.4f} | "
          f"TrainAcc {train_acc*100:5.1f}% | ValAcc {val_acc*100:5.1f}%")

Epoch 01 | Loss 0.6886 | TrainAcc  67.5% | ValAcc  57.5%
Epoch 02 | Loss 0.6403 | TrainAcc  76.9% | ValAcc  57.5%
Epoch 03 | Loss 0.6013 | TrainAcc  83.8% | ValAcc  55.0%
Epoch 04 | Loss 0.5633 | TrainAcc  85.0% | ValAcc  52.5%
Epoch 05 | Loss 0.5143 | TrainAcc  89.4% | ValAcc  52.5%
Epoch 06 | Loss 0.4718 | TrainAcc  91.2% | ValAcc  55.0%
Epoch 07 | Loss 0.4053 | TrainAcc  92.5% | ValAcc  55.0%
Epoch 08 | Loss 0.3445 | TrainAcc  94.4% | ValAcc  60.0%
Epoch 09 | Loss 0.2848 | TrainAcc  96.9% | ValAcc  55.0%
Epoch 10 | Loss 0.2313 | TrainAcc  97.5% | ValAcc  47.5%
Epoch 11 | Loss 0.1753 | TrainAcc  98.8% | ValAcc  50.0%
Epoch 12 | Loss 0.1328 | TrainAcc  99.4% | ValAcc  47.5%


In [10]:
def predict(text: str):
    model.eval()
    x = torch.tensor([encode(text, stoi)], dtype=torch.long)
    lengths = torch.tensor([x.size(1)], dtype=torch.long)
    x, lengths = x.to(DEVICE), lengths.to(DEVICE)
    with torch.no_grad():
        logits = model(x, lengths)
        prob = torch.softmax(logits, dim=1)
        pred = prob.argmax(dim=1).item()
    label = "positivo" if pred == 1 else "negativo"
    return label, prob.squeeze().cpu().tolist()

testes = [
    "o atendimento foi maravilhoso e eu recomendo",
    "produto ruim e frustrante",
    "superou minhas expectativas, muito bom",
    "abaixo das expectativas e não funcionou"
]
for t in testes:
    lbl, pr = predict(t)
    print(f"'{t}' -> {lbl} (probs={pr})")

'o atendimento foi maravilhoso e eu recomendo' -> positivo (probs=[0.280742347240448, 0.719257652759552])
'produto ruim e frustrante' -> negativo (probs=[0.9387591481208801, 0.06124088168144226])
'superou minhas expectativas, muito bom' -> positivo (probs=[0.022187907248735428, 0.9778121113777161])
'abaixo das expectativas e não funcionou' -> negativo (probs=[0.9822425842285156, 0.017757412046194077])


In [11]:
predict("uma bela porcaria")

('negativo', [0.509283185005188, 0.4907167851924896])

In [12]:
predict("uma porcaria")

('positivo', [0.4420820474624634, 0.5579179525375366])

In [13]:
predict("muito bom, nota zero")

('positivo', [0.01232894416898489, 0.9876710176467896])

In [14]:
predict("ficou muito ruim")

('negativo', [0.7909395098686218, 0.2090604305267334])

In [15]:
predict("o melhor")

('negativo', [0.623339831829071, 0.37666022777557373])